In [1]:
import sed_vis
import dcase_util
import sys
sys.path.append('../')
from trainer import SED

import torch
import yaml
from encoder import ManyHotEncoder
from utils import classes_labels
from model import CRNN
import pandas as pd

In [2]:
with open("../params.yaml", "r") as f:
    config = yaml.safe_load(f)

encoder = ManyHotEncoder(
    list(classes_labels.keys()),
    audio_len=config["data"]["audio_max_len"],
    frame_len=config["feats"]["n_filters"],
    frame_hop=config["feats"]["hop_length"],
    net_pooling=config["data"]["net_subsample"],
    fs=config["data"]["fs"],
)

sed = SED(config, encoder=encoder, sed=CRNN(**config["net"]))
print(sed.state_dict().keys())

path = '../DvcLiveLogger/dvclive_run/checkpoints/epoch=27-step=23352.ckpt'
state_dict = torch.load(path)["state_dict"]
sed.load_state_dict(state_dict)

odict_keys(['sed.cnn.cnn.conv0.weight', 'sed.cnn.cnn.conv0.bias', 'sed.cnn.cnn.batchnorm0.weight', 'sed.cnn.cnn.batchnorm0.bias', 'sed.cnn.cnn.batchnorm0.running_mean', 'sed.cnn.cnn.batchnorm0.running_var', 'sed.cnn.cnn.batchnorm0.num_batches_tracked', 'sed.cnn.cnn.conv1.weight', 'sed.cnn.cnn.conv1.bias', 'sed.cnn.cnn.batchnorm1.weight', 'sed.cnn.cnn.batchnorm1.bias', 'sed.cnn.cnn.batchnorm1.running_mean', 'sed.cnn.cnn.batchnorm1.running_var', 'sed.cnn.cnn.batchnorm1.num_batches_tracked', 'sed.cnn.cnn.conv2.weight', 'sed.cnn.cnn.conv2.bias', 'sed.cnn.cnn.batchnorm2.weight', 'sed.cnn.cnn.batchnorm2.bias', 'sed.cnn.cnn.batchnorm2.running_mean', 'sed.cnn.cnn.batchnorm2.running_var', 'sed.cnn.cnn.batchnorm2.num_batches_tracked', 'sed.cnn.cnn.conv3.weight', 'sed.cnn.cnn.conv3.bias', 'sed.cnn.cnn.batchnorm3.weight', 'sed.cnn.cnn.batchnorm3.bias', 'sed.cnn.cnn.batchnorm3.running_mean', 'sed.cnn.cnn.batchnorm3.running_var', 'sed.cnn.cnn.batchnorm3.num_batches_tracked', 'sed.cnn.cnn.conv4.weigh

<All keys matched successfully>

In [9]:
ref_df = pd.read_csv(config["data"]["strong_tsv"], sep = '\t')
ref_df = ref_df[ref_df['filename'] == 'Y--dr8rXrv8k_23.000_33.000.wav'].reset_index().drop(columns = ['index'])
ref_df

,filename,onset,offset,event_label
0,Y--dr8rXrv8k_23.000_33.000.wav,1.667,2.657,Speech
1,Y--dr8rXrv8k_23.000_33.000.wav,0.000,0.541,Speech
2,Y--dr8rXrv8k_23.000_33.000.wav,2.849,3.480,Cat
3,Y--dr8rXrv8k_23.000_33.000.wav,0.692,2.529,Cat
4,Y--dr8rXrv8k_23.000_33.000.wav,4.271,4.558,Cat
5,Y--dr8rXrv8k_23.000_33.000.wav,4.798,8.528,Cat
6,Y--dr8rXrv8k_23.000_33.000.wav,3.664,3.991,Cat


In [ ]:

# Load audio signal first
audio_container = dcase_util.containers.AudioContainer().load(
    'tests/data/a001.wav'
)

# Load event lists
reference_event_list = dcase_util.containers.MetaDataContainer().load(
    'tests/data/a001.ann'
)
estimated_event_list = dcase_util.containers.MetaDataContainer().load(
    'tests/data/a001_system_output.ann'
)

event_lists = {
    'reference': reference_event_list, 
    'estimated': estimated_event_list
}

# Visualize the data
vis = sed_vis.visualization.EventListVisualizer(event_lists=event_lists,
                                                audio_signal=audio_container.data,
                                                sampling_rate=audio_container.fs)
vis.show()